In [5]:
import geopandas as gpd
import glob
import pandas as pd
import rasterio
from rasterio.mask import mask
#import arcpy
##from arcpy import env
#from arcpy.sa import *

In [19]:
geo_file = gpd.read_file("C:/repos/satellite-images-nso-datascience/annotations/2019_03_08_annotaties_coepelduynen_2.gpkg")

In [8]:
geo_file = geo_file.explode()

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  """Entry point for launching an IPython kernel.


In [34]:
for index, row in geo_file.iterrows():
     print(row["label"])
     polyPoints =[arcpy.Point(pt[0], pt[1]) for pt in list(row['geometry'].exterior.coords)]
        

Zand
struweel
bos
grass
Zand
asfalt pad
schelpenpad
struweel
Hoog gras
asfalt weg
Laag gras
Zand
Zand


In [35]:
# Arcpy way of cropping data.
#env.workspace = "E:/data/coepelduynen/"

# Set local variables
#inRaster = "E:/data/coepelduynen/20190308_111644_SV1-01_50cm_RD_11bit_RGBI_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif"

#for index, row in geo_file.iterrows():
    # Execute ExtractByPolygon
#    polyPoints =[arcpy.Point(pt[0], pt[1]) for pt in list(row['geometry'].exterior.coords)]
#    extPolygonOut = ExtractByPolygon(inRaster, polyPoints, "INSIDE")

    # Save the output 
#   extPolygonOut.save("E:/data/20190308_"+row["label"]+"_"+str(index)+"_test_arcgis.tif")

In [14]:
df = pd.DataFrame([])

In [22]:
src = rasterio.open("E:/data/coepelduynen/20190308_111644_SV1-01_50cm_RD_11bit_RGBI_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif")
print('raster path opened')
for index, row in geo_file.iterrows():
        
        # Change the crs to rijks driehoek, because all the satelliet images are in rijks driehoek
        if geo_file.crs != 'epsg:28992':
                geo_file = geo_file.to_crs(epsg=28992)

        out_image, out_transform = mask(src,row['geometry'], crop=True)
        out_meta = src.profile.copy()

        out_meta.update({
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        print('convert to RD')

        df_row = pd.DataFrame([ band.flatten()  for band in out_image]).transpose()
        df_row.columns = ["r","g","b","i","ndvi","height"]
        df_row['label'] = row["label"]
        df = df.append(df_row)
        #with rasterio.open("E:/data/coepelduynen_annotations_test_one_zand.tif", "w", **out_meta) as dest:
        #        dest.write(out_image)
        #        dest.close()


raster path opened
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD
convert to RD


In [23]:
df

,0,1,2,3,4,5,label
0,0,0,0,0,0,0,Zand
1,0,0,0,0,0,0,Zand
2,0,0,0,0,0,0,Zand
3,0,0,0,0,0,0,Zand
4,0,0,0,0,0,0,Zand
...,...,...,...,...,...,...,...
1147,0,0,0,0,0,0,Zand
1148,0,0,0,0,0,0,Zand
1149,0,0,0,0,0,0,Zand
1150,0,0,0,0,0,0,Zand


In [9]:
df = pd.DataFrame([ band.flatten()  for band in out_image]).transpose()

In [10]:
df.columns = ["r","g","b","i","ndvi","height"]

In [12]:
df['label'] = "Zand"

In [13]:
df

,r,g,b,i,ndvi,height,label
0,0,0,0,0,0,0,Zand
1,0,0,0,0,0,0,Zand
2,0,0,0,0,0,0,Zand
3,0,0,0,0,0,0,Zand
4,0,0,0,0,0,0,Zand
...,...,...,...,...,...,...,...
2515,0,0,0,0,0,0,Zand
2516,0,0,0,0,0,0,Zand
2517,0,0,0,0,0,0,Zand
2518,0,0,0,0,0,0,Zand


In [7]:
for index, row in gdf.iterrows():
    print(row['label'])
    for pt in list(row['geometry'].exterior.coords): 
        print(pt)

Zand
(88354.11587106907, 470658.3259087265)
(88388.96600318937, 470649.7911824929)
(88357.67200699971, 470643.74575141084)
(88354.11587106907, 470658.3259087265)
water
(88327.08923799616, 470587.91441729973)
(88355.18271184826, 470591.1149396373)
(88329.22291955455, 470564.7995337505)
(88327.08923799616, 470587.91441729973)
Laag gras
(88496.00569470177, 470514.30240353534)
(88499.20621703935, 470471.9843859607)
(88524.81039573999, 470506.1232908949)
(88496.00569470177, 470514.30240353534)


In [8]:
gdf['geometry'][0]

0  0    POLYGON ((88354.116 470658.326, 88388.966 4706...
Name: geometry, dtype: geometry

In [16]:
geom = gdf['geometry'][0]

In [15]:
with fiona.open("./score_model/annotations/sample_annotations.geojson") as f:
    geom_bounds = f.bounds
    geom=f['geometry']

TypeError: 'NoneType' object is not subscriptable

In [17]:
geom

0  0    POLYGON ((88354.116 470658.326, 88388.966 4706...
Name: geometry, dtype: geometry

In [18]:

all_data=[]
for tif_filename in glob.glob("E:/data/coepelduynen/2022*ndvi_height.tif"):
    with Raster(tif_filename) as raster_obj:
        raster_subset = raster_obj.read(bounds=geom_bounds) 
        polygon_mask = rasterio.features.geometry_mask(geometries=[geom],
                                               out_shape=(raster_subset.shape[0], 
                                                          raster_subset.shape[1]),
                                               transform=raster_subset.affine,
                                               all_touched=False,
                                               invert=True)

        all_data.append(raster_subset.array[polygon_mask])

c:\ProgramData\Anaconda3\lib\site-packages\rasterio\features.py:318: ShapeSkipWarning: Invalid or empty shape {'type': 'FeatureCollection', 'features': [{'id': '(0, 0)', 'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': (((88354.11587106907, 470658.3259087265), (88388.96600318937, 470649.7911824929), (88357.67200699971, 470643.74575141084), (88354.11587106907, 470658.3259087265)),)}, 'bbox': (88354.11587106907, 470643.74575141084, 88388.96600318937, 470658.3259087265)}], 'bbox': (88354.11587106907, 470643.74575141084, 88388.96600318937, 470658.3259087265)} at index 0 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)


ValueError: No valid geometry objects found for rasterize